In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

from datetime import datetime, timedelta
import time

In [2]:
print('Setting up data generator...')
train_dir = '../../data_v1_adjusted/train'
valid_dir = '../../data_v1_adjusted/val'
test_dir = '../../data_v1_adjusted/test'

BATCH_SIZE = 64
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
base_learning_rate = 0.0003
initial_epochs = 1
fine_tune_epochs = 1
pred_thresholds = 0.4, 0.5, 0.6, 0.7, 0.8, 0.9
fine_tune_at = 100
drop_out = 0.8

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
train_labels = tf.concat([y for x, y in train_dataset], axis=0)

validation_dataset = tf.keras.utils.image_dataset_from_directory(valid_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)
valid_labels = tf.concat([y for x, y in validation_dataset], axis=0)

test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                           batch_size=BATCH_SIZE,
                                                           image_size=IMG_SIZE)
test_labels = tf.concat([y for x, y in test_dataset], axis=0)

classes = train_dataset.class_names

Setting up data generator...
Found 8431 files belonging to 5 classes.
Found 1052 files belonging to 5 classes.
Found 1060 files belonging to 5 classes.


In [3]:
print('configuring data training performance...')
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

configuring data training performance...


In [4]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    block_type=None,
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=True,
)(input_node)
output_node = ak.ClassificationHead(len(classes), loss=tf.keras.losses.CategoricalCrossentropy())(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=False, max_trials=10
)
clf.fit(train_dataset, validation_data=validation_dataset, epochs=10)

Trial 10 Complete [00h 07m 29s]
val_loss: 1.565567970275879

Best val_loss So Far: 1.565567970275879
Total elapsed time: 03h 34m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
132/132 [==============================] - 37s 258ms/step - loss: 1.5854 - accuracy: 0.2994 - val_loss: 1.6011 - val_accuracy: 0.3118
Epoch 2/10
132/132 [==============================] - 41s 291ms/step - loss: 1.5727 - accuracy: 0.3118 - val_loss: 1.5720 - val_accuracy: 0.3118
Epoch 3/10
132/132 [==============================] - 43s 306ms/step - loss: 1.5685 - accuracy: 0.3117 - val_loss: 1.5668 - val_accuracy: 0.3118
Epoch 4/10
132/132 [==============================] - 43s 301ms/step - loss: 1.5690 - accuracy: 0.3117 - val_loss: 1.5678 - val_accuracy: 0.3118
Epoch 5/10
132/132 [==============================] - 44s 315ms/step - loss: 1.5683 - accuracy: 0.3117 - val_loss: 1.5665 - val_accuracy: 0.3118
Epoch 6/10
132/132 [==============================] - 45s 320ms/step - loss: 1.5677 - accuracy: 0.3117 -

In [5]:
model_res = clf.export_model()
model_res.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 224, 224, 3)      0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 224, 224, 3)      7         
 n)                                                              
                                                                 
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 32)      9248  